# Google Earth Engine - Python API - Vegetation Change Methods - Case Study - Tolima Department, Columbia - 2017 Semester A Growing Season

## Environment Setup

In [1]:
# Import packages
import ee
import geemap as gm
import vegetation_change as vc

In [2]:
# Initialze GEE Python API; authenticate if necessary
try:
    ee.Initialize()
    
except Exception as error:
    ee.Authenticate()
    ee.Initialize()

In [3]:
# Define output folder (GEE username + folder location)
gee_username = "calekochenour"
gee_asset_folder = "vegetation-change" 

## Data Preparation

In [4]:
# Create FeatureCollections for study area
study_area_boundary = ee.FeatureCollection(
    f'users/{gee_username}/{gee_asset_folder}/drtt_study_area_boundary')

study_area_canals = ee.FeatureCollection(
    f'users/{gee_username}/{gee_asset_folder}/drtt_study_area_canals')

In [5]:
# Load and clip imagery for 2017 Semester A growing season
peak_green = ee.Image(
    'LANDSAT/LC08/C01/T1_SR/LC08_008057_20170602').clip(study_area_boundary)

post_harvest = ee.Image(
    'LANDSAT/LC08/C01/T1_SR/LC08_008057_20170906').clip(study_area_boundary)

# Create ImageCollection for Peak Green and Post-Harvest
collection = ee.ImageCollection([peak_green, post_harvest])

## Data Processing

In [6]:
# Compute NDVI difference raster for Peak Green to Post-Harvest
ndvi_diff = vc.ndvi_diff_landsat8(collection, 1, 0)

In [7]:
# Define NDVI thresholds for classification;
#  indices 0/1 identify min/max for primary class;
#  indices 2/3 identify min/max for secondary class
ndvi_change_thresholds = [-2.0, -0.5, -0.5, -0.35]

In [8]:
# Segment, classify, and extract features
change_features = vc.segment_snic(
    ndvi_diff, study_area_boundary, ndvi_change_thresholds)

In [9]:
# Convert rasters to vectors
change_primary_vector = vc.raster_to_vector(
    change_features.get('primary'), study_area_boundary)

change_secondary_vector = vc.raster_to_vector(
    change_features.get('secondary'), study_area_boundary)

## Data Export

In [10]:
# Define output GEE Asset names
change_primary_asset_name = f'users/{gee_username}/{gee_asset_folder}/vegetation_change_primary'
change_secondary_asset_name = f'users/{gee_username}/{gee_asset_folder}/vegetation_change_secondary'

# Check if GEE Asset already exists prior to export; primary change
if (change_primary_asset := ee.FeatureCollection(change_primary_asset_name)):

    # Skip export
    print(
        f"GEE Asset ID '{change_primary_asset_name}' already exists. Skipping export...")

else:
    # Export vectors to GEE Asset
    change_primary_export = vc.export_vector(
        vector=change_primary_vector,
        description='Primary Change',
        output_name=change_primary_asset_name,
        output_method='asset')

# Check if GEE Asset already exists prior to export; secondary change
if (change_secondary_asset := ee.FeatureCollection(change_secondary_asset_name)):

    # Skip export
    print(
        f"GEE Asset ID '{change_secondary_asset_name}' already exists. Skipping export...")

else:
    # Export vectors to GEE Asset
    change_secondary_export = vc.export_vector(
        vector=change_secondary_vector,
        description='Secondary Change',
        output_name=change_secondary_asset_name,
        output_method='asset')

GEE Asset ID 'users/calekochenour/vegetation-change/vegetation_change_primary' already exists. Skipping export...
GEE Asset ID 'users/calekochenour/vegetation-change/vegetation_change_secondary' already exists. Skipping export...


## Data Visualization

In [11]:
# Create map for visualization
change_map = gm.Map()
change_map.setOptions('SATELLITE')

# Center map to study area
change_map.setCenter(-75.0978, 3.7722, 12)

# Add pre-change and post-change images to map, RGB and CIR
change_map.addLayer(
    peak_green,
    vc.vis_params_rgb,
    'Landsat 8 - RGB - 2017 - Semester A - Peak Green - Pre-Change')

change_map.addLayer(
    post_harvest,
    vc.vis_params_rgb,
    'Landsat 8 - RGB - 2017 - Semester A - Post Harvest - Post-Change')

change_map.addLayer(
    peak_green,
    vc.vis_params_cir,
    'Landsat 8 - CIR - 2017 - Semester A - Peak Green - Pre-Change')

change_map.addLayer(
    post_harvest,
    vc.vis_params_cir,
    'Landsat 8 - CIR - 2017 - Semester A - Post Harvest - Post-Change')

# Add NDVI difference to map, both uncategorized and categorized with SLD
change_map.addLayer(
    ndvi_diff,
    vc.vis_params_ndvi_diff,
    "NDVI Difference - Continuous - 2017 - Semester A - Peak Green to Post-Harvest - Pre to Post-Change")

change_map.addLayer(
    ndvi_diff.sldStyle(vc.vis_params_ndvi_sld),
    {},
    'NDVI Difference - 5 Intervals/Categories - 2017 - Semester A - Peak Green to Post-Harvest - Pre to Post-Change')

# Add classified/extracted rasters (primary and secondary)
change_map.addLayer(
    change_features.get('primary'),
    {'palette': ['green']},
    "Classified Clusters - Raster - Primary Change - 2017 Semester A - Peak Green to Post-Harvest - Pre to Post-Change")

change_map.addLayer(
    change_features.get('secondary'),
    {'palette': ['lightgreen']},
    "Classified Clusters - Raster - Secondary Change - 2017 Semester A - Peak Green to Post-Harvest - Pre to Post-Change")

# Add classified/extracted vectors (from workflow internal)
change_map.addLayer(
    change_primary_vector,
    {'color': 'green'},
    "Classified Clusters - Vector - Primary Change - 2017 Semester A - Peak Green to Post-Harvest - Pre to Post-Change")

change_map.addLayer(
    change_secondary_vector,
    {'color': 'lightgreen'},
    "Classified Clusters - Vector - Secondary Change - 2017 Semester A - Peak Green to Post-Harvest - Pre to Post-Change")

# Add classified/extracted vectors (from workflow GEE Asset export)
change_map.addLayer(
    change_primary_asset,
    {'color': 'green'},
    "Classified Clusters - GEE Asset - Primary Change - 2017 Semester A - Peak Green to Post-Harvest - Pre to Post-Change")

change_map.addLayer(
    change_secondary_asset,
    {'color': 'lightgreen'},
    "Classified Clusters - GEE Asset - Secondary Change - 2017 Semester A - Peak Green to Post-Harvest - Pre to Post-Change")

# Add study area boundary and canals to map
empty = ee.Image().byte()

study_area_boundary_vis = empty.paint(
    featureCollection=study_area_boundary, color=1, width=3)

study_area_canals_vis = empty.paint(
    featureCollection=study_area_canals, color=1, width=3)

change_map.addLayer(
    study_area_boundary_vis,
    {'palette': 'FF0000'},
    'Study Area - Boundary')

change_map.addLayer(
    study_area_canals_vis,
    {'palette': 'blue'},
    'Study Area - Canals')

In [12]:
# Display map
change_map

Map(center=[3.7722, -75.0978], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…